# Flask 프레임워크를 이용한 웹 서빙

## Flask 서버 구성
- app.py
- cnn_model.py
- photos-cnn-model.h5
- templates
    - index.html
- static
    - uploads
        - test-salad.jpg

### 1) index.html 

In [6]:
<!DOCTYPE html>
<head>
    <meta charset="utf8">
    <title>CNN 모델 웹서빙</title>
    <style>
	html { 
		font-size: 20px; 
                         font-weight: bold; 
             }
            img {
                         float: left;
                         width: 40%;
                         height: 50%;
		margin: 5px;
		padding: 5px;
                         border: 2px solid #90C;
            }
            p    {
                         clear: both;
            }
    </style>
<body>
    <div class="container">
    <h1>CNN 모델로 여권위변조 이미지 분류</h1>
    <h2>이미지 업로드 및 CNN 예측</h2>
    <hr>
    {% if filename %}<br>
        <div>
            <img src="{{filename}}" width=500 height=450 align=left>
        </div>   
        <br>       
         <p>예측 : {{label}} &nbsp;&nbsp;  정확도 : {{probability}} &nbsp;&nbsp; 칼로리 : {{cal}}</p>          
    {% endif %}

    <form method="post" action="/upload" enctype="multipart/form-data">
        <input type="file" name="file">
        <input type="submit" value="submit">
    </form>
    </div>
</body>
</head>

SyntaxError: invalid syntax (1149990454.py, line 1)

### 2) app.py 

- pip install flask

In [10]:
import os
from flask import Flask, render_template, request
from PIL import Image
import numpy as np
from tensorflow.keras.models import load_model

# Flask 애플리케이션 초기화
app = Flask(__name__)
app.config['UPLOAD_FOLDER'] = 'static/uploads'

# 학습된 CNN 모델 불러오기
model = load_model("./cnn_model.py")
# model = load_model(model_path)
# model.load_weights(weights_path)

# 테스트 이미지 변형을 위한 변수 선언
im_rows = 32  # 이미지의 높이
im_cols = 32  # 이미지의 너비
im_color = 3  # 이미지의 색공간
in_shape = (im_rows, im_cols, im_color)  # 입력 이미지 차원

def predict(filename):
    # 이미지 읽어 들이기
    img = Image.open("./" + filename)
    img = img.convert("RGB")  # 색공간 변환하기
    img = img.resize((im_cols, im_rows))  # 크기 변경하기

    # 3차원으로 데이터 변환하기
    x = np.asarray(img)
    x = x.reshape(-1, im_rows, im_cols, im_color)
    x = x / 255

    # 예측하기
    pre = model.predict([x])[0]
    idx = pre.argmax()
    per = round(float(pre[idx] * 100), 3)

    return idx, per

# 이미지 업로드 및 분류 페이지
@app.route('/')
@app.route('/index')
def index():
    return render_template('index.html')

@app.route('/upload', methods=['POST'])
def upload():
    file = request.files['file']
    filename = file.filename
    file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
    img_src = os.path.join(app.config['UPLOAD_FOLDER'], filename)

    label, prob = predict(img_src)
    
    return render_template('index.html', filename=img_src, label=label, probability=prob)

if __name__ == '__main__':
    app.run('127.0.0.1', port=5000)



OSError: Unable to open file (file signature not found)

In [1]:
from flask import Flask, request, jsonify

app = Flask(__name__)

@app.route('/process-images', methods=['POST'])
def process_images():
    try:
        # MultiValueMap으로 전송된 이미지 데이터를 받아옵니다.
        image_map = request.files.getlist('image')
        
        # 이미지 데이터를 처리하는 작업을 수행합니다.
        # 예를 들어, 각 이미지를 순회하며 분석 또는 처리 작업을 수행합니다.

        results = []

        for image_data in image_map:
            # 여기에서 각 이미지 데이터를 처리하고 결과를 생성합니다.
            # 이 결과를 JSON 형식으로 저장합니다.
            result = {'message': '이미지가 성공적으로 처리되었습니다.'}
            results.append(result)

        return jsonify(results), 200  # 200은 성공적인 응답을 나타냅니다.
    except Exception as e:
        return jsonify({'error': str(e)}), 500  # 500은 내부 서버 오류를 나타냅니다.

if __name__ == '__main__':
    app.run(host='localhost', port=5001, debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://localhost:5001
Press CTRL+C to quit
 * Restarting with stat
Traceback (most recent call last):
  File "/Users/ahyeonlee/Library/Python/3.10/lib/python/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/Users/ahyeonlee/Library/Python/3.10/lib/python/site-packages/traitlets/config/application.py", line 1042, in launch_instance
    app.initialize(argv)
  File "/Users/ahyeonlee/Library/Python/3.10/lib/python/site-packages/traitlets/config/application.py", line 113, in inner
    return method(app, *args, **kwargs)
  File "/Users/ahyeonlee/Library/Python/3.10/lib/python/site-packages/ipykernel/kernelapp.py", line 678, in initialize
    self.init_sockets()
  File "/Users/ahyeonlee/Library/Python/3.10/lib/python/site-packages/ipykernel/kernelapp.py", line 317, in init_sockets
    self.shell_port = self._bind_socket(self.shell_socket, self.shell_port)
  File "/Users/ahyeonlee/Library/Python/3.10/lib/python/site-packages/ipykern

SystemExit: 1

/Users/ahyeonlee/Library/Python/3.10/lib/python/site-packages/IPython/core/interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [1]:
f = "USA_PASSPORT_2020_UV365"
if 'UV' in f:
    print("있다 ")
else:
    print("없다")

있다 
